In [35]:
#export
import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F

In [36]:
#export
class ResNetEncoder(nn.Module):
    def __init__(self, model_name, embed_dim=512, drop_p=0.3, pretrained=True):
        """Load the pretrained ResNet-152 and replace top fc layer."""
        super(ResNetEncoder, self).__init__()
        self.drop_p = drop_p
        
        if model_name=='resnet18':
            resnet = models.resnet18(pretrained=pretrained)
        elif model_name=='resnet34':
            resnet = models.resnet34(pretrained=pretrained)
        elif model_name=='resnet50':
            resnet = models.resnet50(pretrained=pretrained)
        elif model_name=='resnet152':
            resnet = models.resnet152(pretrained=pretrained)
        else:
            raise f"Model {model_name} is not available"
            
        modules = list(resnet.children())[:-1]      # delete the last fc layer.
        self.resnet = nn.Sequential(*modules)
        self.fc1 = nn.Linear(resnet.fc.in_features, 1024)
        self.bn1 = nn.BatchNorm1d(1024, momentum=0.01)
        self.fc2 = nn.Linear(1024, 768)
        self.bn2 = nn.BatchNorm1d(768, momentum=0.01)
        self.fc3 = nn.Linear(768, embed_dim)
        
    def forward(self, x_3d):
        cnn_embed_seq = []
        for t in range(x_3d.size(1)):
            # ResNet CNN
            with torch.no_grad():
                x = self.resnet(x_3d[:, t, :, :, :])  # ResNet
                x = x.view(x.size(0), -1)             # flatten output of conv

            # FC layers
            x = self.bn1(self.fc1(x))
            x = F.relu(x)
            x = self.bn2(self.fc2(x))
            x = F.relu(x)
            x = F.dropout(x, p=self.drop_p, training=self.training)
            x = self.fc3(x)

            cnn_embed_seq.append(x)

        # swap time and sample dim such that (sample dim, time dim, CNN latent dim)
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=0).transpose_(0, 1)
        # cnn_embed_seq: shape=(batch, time_step, input_size)

        return cnn_embed_seq


class RNNDecoder(nn.Module):
    def __init__(self, embed_dim=512, drop_p=0.3, num_classes=2):
        super(RNNDecoder, self).__init__()
        self.input_size = embed_dim
        self.drop_p = drop_p
        self.num_classes = num_classes

        self.LSTM = nn.LSTM(
            input_size=self.input_size,
            hidden_size=512,        
            num_layers=3,       
            batch_first=True,       # input and output will have batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, self.num_classes)

    def forward(self, input):
        
        self.LSTM.flatten_parameters()
        x, (h_n, h_c) = self.LSTM(input, None)  
        """ h_n shape (n_layers, batch, hidden_size), h_c shape (n_layers, batch, hidden_size) """ 
        """ None represents zero initial hidden state. RNN_out has shape=(batch, time_step, output_size) """

        # FC layers
        x = self.fc1(x[:, -1, :])   # choose RNN_out at the last time step
        x = F.relu(x)
        x = F.dropout(x, p=self.drop_p, training=self.training)
        output = self.fc2(x)
        return output


In [10]:
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu")  

In [11]:
# EncoderCNN architecture
res_size = 224        # ResNet image size
embed_dim=512

# training parameters
k = 2        # number of target category
epochs = 120        # training epochs
batch_size = 4
learning_rate = 1e-3
log_interval = 10   # interval for displaying training info


In [22]:
cnn_encoder = ResNetEncoder(model_name='resnet34',embed_dim=embed_dim, drop_p=0.0).to(device)
rnn_decoder = RNNDecoder(embed_dim=embed_dim, drop_p=0.0, num_classes=k).to(device)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /Users/nikitakaraev/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:13<00:00, 6543553.33it/s]


In [28]:
inp = torch.randn((2,30,3,224,224))
inp.size()

torch.Size([2, 30, 3, 224, 224])

In [29]:
encoded=cnn_encoder(inp)
encoded.size()

torch.Size([2, 30, 512])

In [30]:
decoded = rnn_decoder(encoded_inp)
decoded.size()

torch.Size([2, 2])

In [31]:
decoded

tensor([[0.0426, 0.0011],
        [0.0419, 0.0008]], grad_fn=<AddmmBackward>)

In [37]:
!python notebook2script.py 01_model.ipynb

Converted 01_model.ipynb to exp/nb_01.py
